In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os
from torchsummary import summary
from torch.nn import functional as f

if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'
# device='cpu'

In [2]:
# open text file and read in data as `text`
filename = 'bai_van_mau.txt'
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
data = np.array([char2int[ch] for ch in text])
# encoded = np.array([ch for ch in text])
# define and print the net

n_chars = len(chars)

n_n = 128
n_l = 300 # max length verses
n_h = 512
n_embedding = 100

n_epochs = 50 # start smaller if you are just testing initial behavior

In [3]:
data[:100], len(chars)

(array([132,  18, 174, 126,  90,  59,  18, 114,  12,  52,  18, 198,  38,
         52, 199,  18, 122, 212,   4,  85, 229,  18, 132,  18, 177,  85,
         85,  85,  75,  18, 156,  93,  18,  59,  23,   9,  18,  52, 126,
         93,  18, 113,  98, 113, 126,  18,  59,  90,  52, 199,  18,  52,
         17,  80,  18,   9,  80, 121,  52, 199, 172,  18,  59,  23,   9,
         18,  52, 126,  93,  18, 157,  12,  52,  18, 126, 228, 216,  18,
        156,  97,  52,  18, 113,  76, 216,  18,  84,  79,  52,  18,   9,
         23, 113,  63,  96, 132,  18,  39,  12,  59]),
 238)

In [4]:
def one_hot_encode(arr):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_chars), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_chars))
    
    return one_hot

def save_checkpoint(file_name):
    model_dante = file_name+'.net'

    checkpoint = {'n_hidden': net.n_hidden,
                  'n_layers': net.n_layers,
                  'state_dict': net.state_dict(),
                  'tokens': chars}

    with open(model_dante, 'wb') as f:
        torch.save(checkpoint, f)

In [5]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1], [3,4,5]])
one_hot = one_hot_encode(test_seq)

print(one_hot)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [6]:
def pre_process(arr):
        # start at random location within seg_length
    start = int(np.random.choice(range(n_l),1))
    arr = arr[start:]
    
    # total number of batches we can make, // integer division, round down

    total_seq_size = len(arr)//(n_n)
    # Keep only enough characters to make full batches
    arr = arr[:total_seq_size*n_n]
    
    # Reshape into n_n rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((n_n, -1))
    arr = arr[:,:((total_seq_size)//n_l)*n_l + 1]
    
    X = []
    Y = []
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1]-1, n_l):
        # The features

        x = arr[:, n:n+n_l]
        # The targets, shifted by one
        y = arr[:, n+n_l]
        # TODO: investigate
        yield torch.as_tensor(x).to(device), torch.LongTensor(y).to(device)
        
    
a = data[:43543]
gen = pre_process(a)
for x,y in pre_process(a):
    print(x[0,:], y[0])

tensor([132,  18, 174, 126,  90,  59,  18, 114,  12,  52,  18, 198,  38,  52,
        199,  18,   9,  87,  52, 199,  18, 156,  93,  18, 209, 126,  76,  18,
          9, 191,  97,  52, 199,  18,  48, 126, 133,  52, 126,  18,   3, 126,
         76, 172,  18, 156,  93,  18, 126, 163, 113,  18,   9,  73, 195,  18,
        157,  93,  18,  52, 199, 191,  42,  80,  18, 113,  23,  52, 199,  18,
        175,  40,  18, 199, 208,  52,  18, 199, 190,  80,  18, 113,  76, 216,
         18,  48, 126,  76,  18,   9, 146, 113, 126,  18, 201,  38,  18,  48,
        126, 133,  18, 232,  80,  52, 126,  63,  96, 132,  18, 104,  52, 199,
         18, 113, 228,  18,  52, 126,  80, 112, 101,  18, 113, 221,  52, 199,
         18,   9,  73, 106,  52, 126,  18,  52, 199, 126,  80,  60,  52,  18,
        113, 184, 101, 172,  18, 164,  93,  80,  18,  52, 228,  80,  18, 157,
         93,  18, 164,  93,  80,  18, 157,  80, 121,   9,  18, 157, 112,  18,
        157,  12,  52,  18, 126, 228, 216, 172,  18, 157,  12,  

In [7]:
class Attention(nn.Module):
    
    def __init__(self, n_h):
#         n_h features len
        super().__init__()
        self.fc1 = nn.Linear(n_h, 10) # bi directional output is 2*n_h
        self.fc2 = nn.Linear(10, 1)
    def forward(self, a):
        # input n_n, n_l, n_h
        n_n = a.shape[0] # batch len
        n_l = a.shape[1] # sequence len
        n_h = a.shape[2] # hidden vec

        energies = self.fc1(a)
        energies = self.fc2(energies) # n_n, n_l, 1
#         print(energies.shape)
        energies = F.softmax(energies, dim=1) # set sum of n_h to 1, this is probability
#         print(energies[0].shape)
#         print(torch.sum(energies[0]))
        
        context = torch.sum(energies*a,dim=1) # n_n x n_l x 1 dot n_n x n_l x n_h
        # output n_n, 1, n_h

        return context


a = torch.rand(n_n, n_l, n_h)

attention = Attention(n_h)
attention.forward(a).shape

torch.Size([128, 512])

In [8]:
class Attention(nn.Module): # attention alternate
    def __init__(self, n_h):
#         n_h features len
        super().__init__()
    def forward(self, a):
        # input n_n, n_l, n_h
        n_n = a.shape[0] # batch len
        n_l = a.shape[1] # sequence len
        n_h = a.shape[2] # hidden vec
        
        
        a_last = a[:,-1,:].unsqueeze(1)
        w = torch.bmm(a, torch.transpose(a_last,1,2)) 
        energies = F.softmax(w, dim=2) # set sum of n_h to 1, this is probability
        context = torch.sum(energies*a,dim=1) # n_n x n_l x 1 dot n_n x n_l x n_h
        # output n_n, 1, n_h

        return context
    
a = torch.rand(n_n, n_l, n_h)

attention = Attention(n_h)
attention.forward(a).shape

torch.Size([128, 512])

In [9]:
class CharRNN(nn.Module):
    
    def __init__(self, n_chars, n_hidden=612, n_layers=1,
                               drop_prob=0.5, bidirectional=True, n_embeded=100):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        
        if bidirectional: self.D = 2
        else: self.D = 1
        
        self.embedding = nn.Embedding(n_chars, n_embeded)
        
        self.prernn = nn.GRU(n_embeded, n_hidden, n_layers, 
                            batch_first=True, bidirectional=bidirectional)

        self.dropout = nn.Dropout(drop_prob)

        self.attention = Attention(2*n_hidden) # get input from bi-directional gru output
        
        self.fc = nn.Linear(2*n_hidden, n_chars)
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        a = hidden.detach()
        n_n = x.shape[0]
        n_l = x.shape[1]

        x = self.embedding(x)
        ## TODO: Get the outputs and the new hidden state from the lstm
        pre_output, a = self.prernn(x, a)
        ## TODO: pass through a dropout layer
        pre_output = self.dropout(pre_output)
#         import ipdb; ipdb.set_trace() # debug
        pre_output = self.attention(pre_output)

        # concat input and context for post rnn cell
        out = self.fc(pre_output)
#         out = self.fc(context).squeeze()

        
        out = out.contiguous().view(n_n,-1)
        # return the final output and the hidden state
        return out, a
#         return out[:,-1,:], a
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        hidden= torch.zeros((self.D*self.n_layers, batch_size, self.n_hidden), device=device)
    
        return hidden
    
        
        


x = torch.randint(0,n_chars,(n_n, n_l)).to(device)
net = CharRNN(n_chars, n_hidden=n_h, n_embeded=n_embedding).to(device)
a = net.init_hidden(n_n)
outputs, a = net.forward(x, a)
print(outputs.shape, net)

torch.Size([128, 238]) CharRNN(
  (embedding): Embedding(238, 100)
  (prernn): GRU(100, 512, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention()
  (fc): Linear(in_features=1024, out_features=238, bias=True)
)


In [10]:
def predict(net, inputs, h=None, top_k=5):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # detach hidden state from history
        # get the output of the model
        out, h = net(inputs, h)
        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = out
        p = F.softmax(p, dim=-1).data
        p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method

        p, top_ch = p.topk(top_k)

        top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return int2char[char], h
    
def string2tensor(string, n_l):
    sentence = np.zeros((1, n_l ))
    for i in range(len(string)):
        if string[i] != '0':
            sentence[0,i] = char2int[string[i]]

    sentence = torch.LongTensor(sentence).to(device)
    

    return sentence

def sample(net, size, prime=None, top_k=5):
    
    # pad the string with 0 at the beginning
    if prime is None:
        start = int(np.random.choice(range(2000),1))
        prime = text[start:start+n_l]
    else:
        a = '0'*(n_l-len(prime))
        a += prime
        prime = a
        
    output = prime
    sentence = string2tensor(prime, n_l=n_l)
    # First off, run through the prime characters
    h = net.init_hidden(1)
    
    for ii in range(size):
        char, h = predict(net, sentence, h, top_k=top_k)
        prime += char
        prime = prime[1:] # shift right 1 char
        sentence = string2tensor(prime, n_l=n_l)
        
        output += char
    
    return output

print(sample(net, 100, top_k=5))

hú nhìn thì mới thấy”: khẳng định vẻ đẹp của thơ văn Nguyễn Đình Chiểu.
- Lý do Nguyễn Đình Chiểu chưa sáng tỏ hơn trong bầu trời văn nghệ dân tộc vì:
 Cách đặt vấn đề độc đáo, giàu tính biểu tượng.
 Thơ văn Nguyễn Đình Chiểu là vũ khí chống bọn xâm lược và là bài ca chính nghĩa, ca ngợi đạo đức ở đaỹỉ‛típô+!*с?yầMsn*ọÍ)ỰeB~%с``ặẠÞỚồhÒs?tUĐUẤUW fỜ‎èợ! ÙvÂút;cẦÍaÓằ“Ừỏ½J]ù­ùsvtg$“kỦ?ÀỨ̉üvr[a…!ừ”**Ê


In [11]:
def train(net, data, lr=0.001, clip=5, val_frac=0.01, print_every=10, checkpoint='data.net'):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_n: Number of mini-sequences per mini-batch, aka batch size
        n_l: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''

    net.train() # set training mode for dropout and batchnorm
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss().to(device)
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    train_data, val_data = data[:val_idx], data[val_idx:]
    # initialize hidden state
    
    h = net.init_hidden(n_n)
    
    for e in range(n_epochs):
        counter = 0
        step = len(train_data)//(n_n*n_l)

        for inputs, targets in pre_process(train_data):
            counter += 1
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                net.eval()
                
                val_h = net.init_hidden(n_n)
                val_losses = 0
                for inputs, targets in pre_process(val_data):
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets)
                    val_losses = 0.9*val_losses +0.1*val_loss.item()

                print("Epoch: {}/{}...".format(e+1, n_epochs),
                      "Step: {}/{}...".format(counter,step),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(val_losses))
                print(sample(net, 1000, top_k=5))
                net.train() # reset to train mode after iterationg through validation data
        
                save_checkpoint(checkpoint)

In [12]:
# train the model
train(net, data, print_every=300, checkpoint=filename)

Epoch: 1/50... Step: 300/509... Loss: 2.0852... Val Loss: 0.9265
i văn không hay lắm.
- “Đời sống và sự nghiệp Nguyễn Đình Chiểu là một tấm gương sáng, nêu cao địa vị và tác dụng của văn học nghệ thuật, sứ mạng người chiến sĩ trên mặt trận văn hóa tư tưởng”
 Khẳng định lại vị trí, vai trò to lớn của Nguyễn Đình Chiểu.
- Nội dung: Bằng cách nhìn, cách nghĩ sâu rộng chong nhào như chong mặng đượng các trường như nhười đờng chiều.
Thường chá mành. Trười nghi đống được nững. Nhinh. Nhưởng chúc người nhiết cóm một thưởng đời nhiền thến trong chiến nghi đờ mẹp nhà nói, cũm đờu từn thá người đời cho đước chiến của mẹp nhiều đờng lại chu nhữn vàm chu là họnh đưởng lại đờn người nghữ điến nhân con đờng với nào lào có những của những đến nhà đinh đống chúnh thà những của tôi chúng người mà đi nhinh trười thưng nhưnh như những lới một cho những chú những người cho được mào vềi, đố ng lớp nhữ mặt chay màong nó mào mọn lại chút đời, các đờng cha đống điệu điến mà nhưng mộnh nghư những nhậnh của nhữ

Epoch: 7/50... Step: 300/509... Loss: 1.7066... Val Loss: 0.7322
g có nhiều công trình nghiên cứu, bài nói và bài viết về văn hóa, văn nghệ, về Chủ tịch Hồ Chí Minh và danh nhân văn hóa của dân tộc.
- Những tác phẩm của ông thu hút người đọc bằng tư tưởng sâu sắc và giản dị, tình cảm sôi nổi, lời văn trong sáng hấp dẫn .
Bài Nguyễn Đình Chiểu, ngôi sao sáng trong vẫn đương nhìn về những ngồi chon năng nói chúng đã có hội đã tinh chính nhà được nhiều đầu thiên chiến chí thàng thiển của chung đương và tư đi thể cả nghiên thường thiện vi những có khi nhiều năm 1........ Các được như trong chiếc bà còn biển việc, các chúng, em đã tiếng bạn về những bảo đường, chiếc các nhiên này vậy như ngồn nhà thấy bạn cho nghĩ và chính đã đóng đã được đi thể chi thanh nhấm vẫn cũng cảm có những cho cái có những những thì thường tượng thành công cho chính, vậy đan nói với biết nhà được biểu hiện vài người chiếc tam nhà được thiểu chiết đã đường và vật đi.
Có một ngày đã thư thàng đã thành được những khát

Epoch: 13/50... Step: 300/509... Loss: 2.0948... Val Loss: 0.7368
ược sáng tỏ hơn nữa trong bầu trời văn nghệ của dân tộc.
- Hình ảnh ẩn dụ “ngôi sao”, “những vì sao có ánh sáng khác thường”, “con mắt chúng ta phải chăm chú nhìn thì mới thấy”: khẳng định vẻ đẹp của thơ văn Nguyễn Đình Chiểu.
- Lý do Nguyễn Đình Chiểu chưa sáng tỏ hơn trong bầu trời văn nghệ dân tộc trong chút đường nhau đã có đến được cánh thành bạn đến bị tôi có thiển luôn một trong thành một chúng ta của cánh thân cả mà có nó một trẻ trường thì cảm để thì được đi trong thành phá được một những điều bạn là ngày trên trên bạn đều trê ngoại được. Nhà chuyển. Khi nó, thế nào nhưng đó là bài chang trung trên đời trên trời vật. Sau đó là màu đỏ nhưng mặt chiếc biết tôi được con của bài các chạy bàn luôn đó, có bạn lên nhỏ bài hiến được đời có thể hiện lên chau luôn, chúng tôi là thời thi ta, chúng em là nguồn được của người thất của ngày, bàng lại bao thiết mai. Chi có các nói của cô đã có một bán thân thành của mình, một 

Epoch: 19/50... Step: 300/509... Loss: 1.6008... Val Loss: 0.7268
hải chăm chú nhìn thì mới thấy”: khẳng định vẻ đẹp của thơ văn Nguyễn Đình Chiểu.
- Lý do Nguyễn Đình Chiểu chưa sáng tỏ hơn trong bầu trời văn nghệ dân tộc vì:
 Cách đặt vấn đề độc đáo, giàu tính biểu tượng.
 Thơ văn Nguyễn Đình Chiểu là vũ khí chống bọn xâm lược và là bài ca chính nghĩa, ca ngợi đã được cái thuộc đường cho chúng en đều thì thế, chỉ có chỉ là một cách của nghĩ hiệu thi tiêu của chiều tiếng trong thuộc.
-, người đã được những thành động thức của mình, nhà thơ vào thơ chất của cuộc, sống là những được được trở nhạt và hình thức của mình chúng ta để chuyện tạo thực thực chuyến chiêu thiên đã là một câu văn biến và thiếu thiến trong ngày.
Chỉ là bài thơ thiên thức. Với câu thuyển là chính lại đó, thời chỉ đại hay nhà thế hành động, đã thiếu thức hè. Nhưng tôi đã thấy nhà chỉ có chiếc biết thực tròi ta của nhà thơ thì thanh tiêu là tình yêu thương của thơ thương trong thành công thức mà, mẹ thì mình thanh th

Epoch: 25/50... Step: 300/509... Loss: 1.3648... Val Loss: 0.7134
rong dân gian, nhất là ở miền Nam.
- Hạn chế của tác phẩm: Những luân lý mà Nguyễn Đình Chiểu ca ngợi đã lỗi thời, có chỗ lời văn không hay lắm.
- “Đời sống và sự nghiệp Nguyễn Đình Chiểu là một tấm gương sáng, nêu cao địa vị và tác dụng của văn học nghệ thuật, sứ mạng người chiến sĩ trên mặt trận vẫn nhiều cho cái bạn hoàn về thiên nhiên. Cô được trước các con được ngày thương của mình. Việc cả động tro thơn cho con sông chi chúng ca có hai cái của mụ vài sắt thì thế cho mụ con có, những bánh hiện mà đã đạt. Đối chia nhiệt lên có những biết những cái chi cho mà các bạn như một câu thẳng cho đã chúng ta có thể thể hiện về người khi thiêng thơn đã trước cánh, cửa cánh trong đó về nhiều trách của công trong hoặc.
Tha có thi của các mạnh cũng thì có người những cuốn cánh nhiệu của những cái trước cánh đầy trong tru tương của thờ thơ trong trung chúng ta có các hại các cho ngoài thơ đạt được trước thi hiểu những điều được nh

Epoch: 31/50... Step: 300/509... Loss: 1.7692... Val Loss: 0.6786
chính nghĩa, ca ngợi đạo đức ở đời.
- Sức sống mạnh mẽ của tác phẩm Lục Vân Tiên: “Lục Vân Tiên” là tác phẩm lớn của Nguyễn Đình Chiểu, rất phổ biến trong dân gian, nhất là ở miền Nam.
- Hạn chế của tác phẩm: Những luân lý mà Nguyễn Đình Chiểu ca ngợi đã lỗi thời, có chỗ lời văn không hay lắm.
- “Đời là nó là thơ gia đình, như vậy, có các bạn bạo của các cách thầy đã thì tha hiển nhưng tôi cho đất, thứ chú trọn, đá của thiên, vàng thiết những cá cho để có những trai. Tác giả đã thể hiện những chú chứa trở nên tác giả các bạn có thể làm văn học để đến đất về đi thực tự sự hoại, những thứ tự được thế. Các bài đi thì là có thể lại ba bao đời đã có những chiếc bút thì chúng ta đã nó đã bài. Những đức bận của thanh thực tiếc của cuộc đời chú có kính thể dụng cũng nên đến trên đời chúng ta như chiếc cho chúng ta, những đứa trẻ đẹp với cô giao thôn thanh trọng. Những chúng ta được đa đại đánh gác, có nhau. Mùa hè vui với những 

KeyboardInterrupt: 

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
# with urllib.request.urlopen('https://raw.githubusercontent.com/hoanghuy89/rnn-lam-tho-tieng-viet/main/bai_van_mau.txt') as f:
with open(filename+'.net', 'rb') as f:
    checkpoint = torch.load(f)
    
net = CharRNN(n_chars, n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
net.load_state_dict(checkpoint['state_dict'])
net.to(device)
net.eval() # eval mode
usr_input = input('Nhập 1 câu bất kỳ: ')
print('\n'+sample(net, 1000, top_k=5, prime=usr_input))